# Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.linear_model import LogisticRegression
from sklearn import datasets

In [32]:
cases = pd.read_csv('Data.nosync/Similar_TM.csv')
cases

,TM_A,TM_B
0,Simoniz,Permanize
1,Magnavoc,Multivox
2,Zirco,Cozirc
3,Platinum Puff,Platinum Plus
4,Maternity Yours,Your Maternity Shop
...,...,...
281,WOODLANDS,WOODLAND
282,THE QUBE,QUBE
283,SUB4,SUB2
284,BlendiT,BLEND IT MILKSHAKES


In [28]:
rejected_cases = pd.read_csv('Data.nosync/Unsimilar_TM.csv')
rejected_cases

,TM_A,TM_B
0,Nutri-cola,coca-cola
1,Up-town,7-up
2,Ocean Spray Cranberries,Ocean Garden Prods
3,THE HYDE PARK DY GIN,HYDE PARK CORNER
4,LUCE DELLA VITE,Arcata Di Luce
...,...,...
66,Lilton,Wilton
67,Nutricia,Nutritea
68,Glenreidh,An Reidhe
69,No Gunk No Junk,No Gunk Just Funk


In [2]:
import unidecode
from fuzzywuzzy import fuzz

from abydos.distance import (IterativeSubString, BISIM, DiscountedLevenshtein, Prefix, LCSstr, MLIPNS, Strcmp95,
MRA, Editex, SAPS, FlexMetric, JaroWinkler, HigueraMico, Sift4, Eudex, ALINE, Covington, PhoneticEditDistance)

from abydos.phonetic import PSHPSoundexFirst, Ainsworth
pshp_soundex_first = PSHPSoundexFirst()
pe = Ainsworth()

In [3]:
iss = IterativeSubString()
bisim = BISIM()
dlev = DiscountedLevenshtein()
prefix = Prefix()
lcs = LCSstr()
mlipns = MLIPNS()
strcmp95 = Strcmp95()
mra = MRA()
editex = Editex()
saps = SAPS()
flexmetric = FlexMetric()
jaro = JaroWinkler(mode='Jaro')
higuera_mico = HigueraMico()
sift4 = Sift4()
eudex = Eudex()
aline = ALINE()
covington = Covington()
phonetic_edit = PhoneticEditDistance()

In [15]:
algos = [iss, bisim, dlev, prefix, lcs, mlipns, strcmp95, mra, editex, saps, flexmetric, jaro, higuera_mico, sift4, eudex,
         aline, covington, phonetic_edit]

algo_names = ['iterativesubstring', 'bisim', 'discountedlevenshtein', 'prefix', 'lcsstr', 'mlipns', 'strcmp95', 'mra',
              'editex', 'saps', 'flexmetric', 'jaro', 'higueramico', 'sift4', 'eudex', 'aline', 'covington',
              'phoneticeditdistance']

In [16]:
from abydos.phones import *

In [17]:
def sum_ipa(name_a, name_b):
    feat1 = ipa_to_features(pe.encode(name_a))
    feat2 = ipa_to_features(pe.encode(name_b))
    score = sum(cmp_features(f1, f2) for f1, f2 in zip(feat1, feat2))/len(feat1)
    return score

In [18]:
import re

In [34]:
def featurize(df):
    if len(df.columns)==3:
        df.columns=['a', 'b', 'target']
    elif len(df.columns)==2:
        df.columns=['a', 'b']
    else:
        df = df.rename(columns={df.columns[0]: 'a', df.columns[1]: 'b' })
        
#     df['TM_A'] = df.apply(lambda row: re.sub(
#         '[^a-zA-Z]+', '', unidecode.unidecode(row['a']).lower().strip()), axis=1)
#     df['TM_B'] = df.apply(lambda row: re.sub(
#         '[^a-zA-Z]+', '', unidecode.unidecode(row['b']).lower().strip()), axis=1)
    
#     df['syll_a'] = df.apply(lambda row: syllable_tokenizer.tokenize(row.name_a), axis=1)
#     df['syll_b'] = df.apply(lambda row: syllable_tokenizer.tokenize(row.name_b), axis=1)
    
#     df['partial'] = df.apply(lambda row: fuzz.partial_ratio(row.syll_a,row.syll_b), axis=1)
#     df['tkn_sort'] = df.apply(lambda row: fuzz.token_sort_ratio(row.syll_a,row.syll_b), axis=1)
#     df['tkn_set'] = df.apply(lambda row: fuzz.token_set_ratio(row.syll_a,row.syll_b), axis=1)
    
#     df['sum_ipa'] = df.apply(lambda row: sum_ipa(row.name_a, row.name_b), axis=1)
    
#     df['pshp_soundex_first'] = df.apply(
#         lambda row: 1 if pshp_soundex_first.encode(row.name_a)==pshp_soundex_first.encode(row.name_b) else 0, axis=1)
    
#     for i, algo in enumerate(algos):
#             df[algo_names[i]] = df.apply(lambda row: algo.sim(row.name_a, row.name_b), axis=1)
            
#     df.drop(['syll_a', 'syll_b'], axis=1, inplace=True)
    return df

In [21]:

from itertools import combinations
import random
random.seed(30)

In [ ]:
# Use combinatorics to generate negative class
all_names = cases.loc[:, 'name_a':'name_b'].values.tolist()
unique_names = list(set([item for items in all_names for item in items]))
alt_pairs = list(zip(cases.name_a, alt_names.name_b))+ list(zip(alt_names.name_b, alt_names.name_a))
comb = list(combinations(unique_names, 2))
non_alt = list(set(comb) - set(alt_pairs))
# Undersample the negative class for 1:4 class imbalance instead of 1:1000 extreme class imbalance
non_alt = pd.DataFrame(random.choices(non_alt, k=70040), columns=['TM_A', 'TM_B'])

In [23]:
df = featurize(cases)
df

NameError: name 'syllable_tokenizer' is not defined

In [31]:
len(cases.shape)

2